   # Python FHIR Demo: Searching for Patient Medications

In [1]:
### Install fhirclient module using sys.executable (the current python)
#import sys
#!{sys.executable} -m pip install --user jupyter fhirclient

In [2]:
### Import required modules / models
import json
from fhirclient import client
import fhirclient.server as s
import fhirclient.models.patient as p
import fhirclient.models.medicationstatement as ms
from IPython.core.display import display, HTML, JSON

In [3]:
### Define a few functions to display output nicely
def pretty(js):
    return json.dumps(js, indent=2)
def prettyprint(x):
    print(pretty(x))

In [4]:
### Set a few options to pass into the FHIRClient object instantiator
settings = {
    'app_id': 'my_web_app',
    'api_base': 'https://vonk.fire.ly/'
#    'api_base': 'https://r3.smarthealthit.org/'
}
### Create an instanct of the fhirclient engine
smart = client.FHIRClient(settings=settings)
### This will do a pre-check to see if any auth is necessary (it's not for this one, so expect 'True')
smart.prepare()


True

In [5]:
###Set the patient id ('pat1' is 'Donald Duck', who we know to have some meds in the demo system)
#pid = '296769f2-5fe9-4997-a521-82c5eb090311'
pid = 'pat2-Fhir3.0'

In [6]:
### Grab our patient directly using the 'read' method on the Patient model and passing in the unique id)
patient = p.Patient.read(pid, smart.server)
### Output the data that comes back (requires knowledge of the structure)
output = "<h2>" + pid + "</h2><p>" + patient.name[0].given[0] + " " + patient.name[0].family + "</p>"

search = ms.MedicationStatement.where({'patient':pid});
meds = search.perform_resources(smart.server)
medslist = {}
serializablemedarray = []
for med in meds:
    serializablemedarray.append(med.as_json());
    mlkey = False;
    # Try to get an existent 'mlkey' with name of medicine from varying structures
    try:
        mlkey = med.medicationCodeableConcept.text
    except:
        mlKey = False
    try:
        mlkey = med.contained[0].code.coding[0].display
    except:
        mlKey = False
    try:
        mlkey = med.medicationCodeableConcept.coding[0].display
    except:
        mlKey = False
    # If the med key was found AND it's not already in our list
    if (mlkey is not False) or (mlkey in medslist):
        pass
    else:
        medslist[mlkey] = {}
        medslist[mlkey]['name'] = mlkey
output = output + "<h4>Medications</h4><ul>";
for m in medslist:
    output = output + "<li>" + medslist[m]['name'] + "</li>" 
output = output + "</ul>";

In [7]:
## Output the json for the patient that we queried
print(json.dumps(patient.as_json(), indent=2))

{
  "id": "pat2-Fhir3.0",
  "meta": {
    "lastUpdated": "2020-10-29T13:14:50.639+00:00",
    "versionId": "1742b699-f152-42ef-8a00-be6fbf7b75bf"
  },
  "text": {
    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">\n      \n      <p>Patient Donald D DUCK @ Acme Healthcare, Inc. MR = 123456</p>\n    \n    </div>",
    "status": "generated"
  },
  "active": true,
  "gender": "other",
  "identifier": [
    {
      "system": "urn:oid:0.1.2.3.4.5.6.7",
      "type": {
        "coding": [
          {
            "code": "MR",
            "system": "http://hl7.org/fhir/v2/0203"
          }
        ]
      },
      "use": "usual",
      "value": "123456"
    }
  ],
  "link": [
    {
      "other": {
        "reference": "https://vonk.fire.ly/Patient/pat1"
      },
      "type": "seealso"
    }
  ],
  "managingOrganization": {
    "display": "ACME Healthcare, Inc",
    "reference": "https://vonk.fire.ly/Organization/1"
  },
  "name": [
    {
      "family": "Donald",
      "given": [
     

In [8]:
## Output the json for the med array that we've been compiling
print(json.dumps(serializablemedarray, indent=2))

[]


In [9]:
## Display some neatly formatted HTML using all that data instead
#display(HTML(output))

In [10]:
## Create an object that has both the patient and medication data in it
mypatient = {}
mypatient["firstname"] = patient.name[0].given[0]
mypatient["lastname"] = patient.name[0].family
mypatient["medications"] = medslist

In [11]:
## Print out one big JSON object to use in our jsFiddle
print(json.dumps(mypatient, indent=2))

{
  "firstname": "Duck",
  "lastname": "Donald",
  "medications": {}
}
